In [1]:
import pandas as pd
import re
import json
import numpy as np
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

third_party_csv_path = r'C:\Users\elien\OneDrive - TU Eindhoven\Internship\Data\third_party_data.csv'
baseline_csv_path = r'C:\Users\elien\baseline_data2.csv'
features_remove_outliers = ['delta_min', 'calories', 'quantity', 'distance', 'step_min', 'move_min', 'MET', 'min_high', 'min_med']

start_date = '2014-01-01'
end_date = '2022-05-22'                       
stepday_min=200
stepday_max=50000
act_min=2
min_days=3
max_days=600

In [2]:
def json_str_to_list(data_act, part_id, dt):
    string = data_act
    string = string.replace("'", '"')
    string = string.replace("None", '"None"')
    string = string.replace("True", '"True"')
    string = string.replace("False", '"False"')

    json_object = json.loads(string)
    activity = pd.DataFrame(json_object)
    
    activity.insert(0, 'participant_id', part_id)
    activity.insert(1, 'datetime', dt)
    js = activity.to_json(orient = 'records')
    json_object = json.loads(js)
    return json_object

def np_vec_json(df):
    vfunc = np.vectorize(json_str_to_list)
    return vfunc(df['data'], df['participant_id'], df['datetime'])

def google_activities(third_party_csv_path):
    third_party_data = pd.read_csv(third_party_csv_path)
    GoogleFitness = third_party_data[third_party_data['data_source'] == 'GoogleFitness']
    GoogleFitness = GoogleFitness.drop(columns=['Unnamed: 0', 'id', 'research_stage'])
    
    #Activity data 
    act_dat = GoogleFitness[GoogleFitness['name'] == 'Activities']
    array = np_vec_json(act_dat)
    array = np.concatenate((array), axis=None)
    activities_participants = pd.json_normalize(array)
    
    ## Add delta time columns to activity dataframe
    activities_participants['end'] = pd.to_datetime(activities_participants['end'], unit='ms')
    activities_participants['start'] = pd.to_datetime(activities_participants['start'], unit='ms')

    #delta_time
    delta_t = activities_participants['end'] - activities_participants['start']
    activities_participants.insert(4, 'delta_time', delta_t)

    #delta_sec
    delta_sec = activities_participants['delta_time'].astype('timedelta64[s]')
    activities_participants.insert(5, 'delta_sec', delta_sec)

    #delta_min
    delta_min = activities_participants['delta_sec'] /60
    activities_participants.insert(6, 'delta_min', delta_min)
    
    activities_participants = activities_participants.drop(columns=['source_id', 'device', 'tracked', 'source_name', 'delta_time'])
    
    return activities_participants

In [13]:
def Mifflin_bmr(weight, height, age, gender):
    bmr = (9.99*weight) + (6.25*height) - (4.92*age) + (166*gender) - 161
    return bmr

def add_baseline_features(act_df, baseline_csv_path):
    baseline = pd.read_csv(baseline_csv_path)
    base = baseline.drop(columns=['Unnamed: 0', 'Date', 'standing_one_min_blood_pressure_systolic', 'lying_blood_pressure_diastolic', 'standing_three_min_blood_pressure_systolic', 'lying_blood_pressure_systolic', 'body_temperature', 'sitting_blood_pressure_pulse_rate', 'standing_one_min_blood_pressure_pulse_rate', 'abdominal', 'body_fat', 'waist', 'standing_three_min_blood_pressure_diastolic', 'dominant_hand', 'sitting_blood_pressure_diastolic', 'standing_three_min_blood_pressure_pulse_rate', 'lying_blood_pressure_pulse_rate', 'standing_one_min_blood_pressure_diastolic', 'sitting_blood_pressure_systolic', 'hips', 'hand_grip_right', 'hand_grip_left', 'bmr'])

    base['bmr_eq'] = Mifflin_bmr(base['weight'].values, base['height'].values, base['age'].values, base['gender'].values) 
    act_base_df = act_df.merge(base.rename(columns={"RegistrationCode": "participant_id"}), on="participant_id", how="inner")
    act_base_df = act_base_df.dropna(subset = ["bmr_eq"])
    return act_base_df

def MET_score(calories, time, weight):
    MET = ((200*calories) / (time*weight)) / 3.5
    return MET

def add_activity_features(act_df, baseline_csv_path):
    act_base_df = add_baseline_features(act_df, baseline_csv_path)
    
    # Move min (>= 30 steps per min) and steps per minute:
    act_base_df['step_min'] = act_base_df['quantity'] / act_base_df['delta_min']
    act_base_df['move_min'] = np.where(act_base_df['step_min'] < 30 , 0, np.where(act_base_df['step_min'] >= 30, act_base_df['delta_min'], 0))

    # MET score (time in min)
    act_base_df['MET'] = MET_score(act_base_df['calories'], act_base_df['delta_min'], act_base_df['weight'])

    # Add heart points - HP's
    # MET value = 3-6: 1 HP_MET
    # MET value = >6: 2 HP_MET
    # step per min = 100-130: 1 HP_step
    # step per min = >130: 2 HP_step
    act_base_df['HP_MET'] = np.where((act_base_df['MET'] >= 3) & (act_base_df['MET'] < 6) , 1, np.where(act_base_df['MET'] >= 6, 2, 0))
    act_base_df['HP_step'] = np.where((act_base_df['step_min'] >= 100) & (act_base_df['step_min'] < 130) , 1, np.where(act_base_df['step_min'] >= 130, 2, 0))

    # HP total (max of HP_MET and HP_step)
    act_base_df['HP'] = (act_base_df['HP_MET'] + act_base_df['HP_step'])
    act_base_df['HP'] = act_base_df['HP'].replace([2], 1)
    act_base_df['HP'] = act_base_df['HP'].replace([3], 2)
    act_base_df['HP'] = act_base_df['HP'].replace([4], 2)
    
    ## Moderate and high intensity minutes:
    # When HP > 2: high intensity minutes
    # When HP == 1: moderate intensity minutes 
    act_base_df['min_high'] = np.where(act_base_df['HP'] == 2 , act_base_df['delta_min'], 0)
    act_base_df['min_med'] = np.where(act_base_df['HP'] == 1 , act_base_df['delta_min'], 0)

    # Multiply HP factor (0,1 or 2) with amount of minutes
    act_base_df['HP_MET'] = act_base_df['HP_MET']*act_base_df['delta_min']
    act_base_df['HP_step'] = act_base_df['HP_step']*act_base_df['delta_min']
    act_base_df['HP'] = act_base_df['HP']*act_base_df['delta_min']

    act_features_df = act_base_df.drop(columns=['delta_sec'])
    return act_features_df

In [4]:
# Outlier: value that is more than 3 standard deviations from the mean

def outlier_removal(df, variable):
    upper_limit = df[variable].mean() + 3 * df[variable].std()
    lower_limit = df[variable].mean() - 3 * df[variable].std()
    return upper_limit, lower_limit

def remove_feature_outliers(act_features_df, features_remove_outliers):
    var = features_remove_outliers
    upper = []
    lower = []

    for v in var:
        upper_limit, lower_limit = outlier_removal(act_features_df, v)
        upper.append(upper_limit)
        lower.append(lower_limit)

    for v in range(len(var)):
        variable = var[v]
        act_features_df = act_features_df[(act_features_df[variable] > lower[v]) & (act_features_df[variable] < upper[v])]

    act_features_outliers_df = act_features_df
    
    return act_features_outliers_df


In [5]:
def aggregate_part_day(act_features_outliers_df, 
                       start_date, end_date, 
                       stepday_min, stepday_max, 
                       act_min):

    table = pd.pivot_table(act_features_outliers_df, values=['HP', 'HP_step', 'HP_MET','MET', 'quantity', 'bmr_eq', 'gender', 'age', 'bmi', 'weight', 'height', 'move_min', 'step_min', 'distance', 'calories', 'activity_name', 'min_high', 'min_med'], index=['participant_id', 'datetime'],
                        aggfunc={'HP': np.sum,
                                 'HP_step': np.sum,
                                 'HP_MET': np.sum,
                                'MET': [np.mean, np.std, min, max],
                                 'quantity': np.sum,
                                'bmr_eq': np.mean,
                                'gender': np.mean,
                                'age': np.mean,
                                'bmi': np.mean,
                                'weight': np.mean,
                                'height': np.mean,
                                'move_min': np.sum,
                                'step_min': np.mean,
                                'distance': np.sum,
                                'calories': np.sum,
                                'activity_name': 'count',
                                'min_high': np.sum,
                                'min_med': np.sum},                          
                             fill_value = 0).sort_index()

    table['activity_score'] = table[('calories', 'sum')] / table[('bmr_eq', 'mean')]
    part_day = table.reset_index()
    part_day.columns = ["_".join((i,j)) for i,j in part_day.columns]
    part_day = part_day.rename(columns={'participant_id_': 'participant_id', 
                                        'datetime_': 'datetime',
                                        'activity_score_': 'activity_score'})

   
    # 1) Data within timerange
    part_day['datetime'] = pd.to_datetime(part_day['datetime'])  
    df0 = part_day[(part_day['datetime'] > start_date) & (part_day['datetime'] <= end_date)]
    
    # 2) Remove outliers activity_score
    upper_limit, lower_limit = outlier_removal(df0, 'activity_score')
    df1 = df0[(df0['activity_score'] > lower_limit) & (df0['activity_score'] < upper_limit)]
    
    # 3) stepday min and max
    df2 = df1[(df1['quantity_sum'] > stepday_min) & (df1['quantity_sum'] <= stepday_max)]
    
    # 4) act_min per day
    df3 = df2[df2['activity_name_count'] > act_min]
    
    act_cleaned_part_day = df3
    
    return act_cleaned_part_day


In [19]:
def aggregate_part(act_cleaned_part_day, min_days, max_days):

    table2 = pd.pivot_table(act_cleaned_part_day, values=['datetime', 'HP_sum', 'HP_MET_sum', 'HP_step_sum', 'MET_max', 'MET_mean', 'MET_min', 'MET_std', 'activity_name_count', 'age_mean', 'bmi_mean', 'bmr_eq_mean', 'calories_sum', 'distance_sum', 'gender_mean', 'height_mean', 'min_high_sum', 'min_med_sum', 'move_min_sum', 'quantity_sum', 'step_min_mean', 'weight_mean', 'activity_score'], 
                           index=['participant_id'],
                        aggfunc={'datetime': 'count',
                                'HP_sum': [np.mean, np.std],
                                 'HP_MET_sum': [np.mean, np.std],
                                 'HP_step_sum': [np.mean, np.std],
                                 'MET_max': np.mean,
                                 'MET_mean': np.mean,
                                 'MET_min': np.mean,
                                 'MET_std': np.mean,
                                 'activity_name_count': [np.mean, np.std],
                                 'age_mean': np.mean,
                                 'bmi_mean': np.mean,
                                 'bmr_eq_mean': np.mean,
                                 'calories_sum': [np.mean, np.std],
                                 'distance_sum': np.mean,
                                 'gender_mean': np.mean,
                                 'height_mean': np.mean,
                                 'min_high_sum': [np.mean, np.std],
                                 'min_med_sum': [np.mean, np.std],
                                 'move_min_sum': [np.mean, np.std],
                                 'quantity_sum' : [np.mean, np.std],
                                 'step_min_mean': [np.mean, np.std],
                                 'weight_mean': np.mean,
                                 'activity_score' : [np.mean, np.std]},                          
                             fill_value = 0).sort_index()

    part_tot = table2.reset_index()
    part_tot.columns = ["_".join((i,j)) for i,j in part_tot.columns]
    part_tot = part_tot.rename(columns={'participant_id_': 'participant_id'})

    
    act_cleaned_part = part_tot[(part_tot['datetime_count'] > min_days) & (part_tot['datetime_count'] <= max_days)]
    
    act_cleaned_part = act_cleaned_part.rename(columns={ 
                                                  'MET_mean_mean': "MET_mean",
                                                 'MET_std_mean': "MET_std",
                                                 'activity_name_count_mean': "activities_count",
                                                 'age_mean_mean': 'age',
                                                 'bmi_mean_mean': 'bmi',
                                                 'bmr_eq_mean_mean': 'bmr_eq',
                                                 'calories_sum_mean': 'calories',
                                                 'calories_sum_mean': 'calories_std',
                                                 'gender_mean_mean': 'gender',
                                                 'height_mean_mean': 'height',
                                                 'move_min_sum_mean': 'move_min_mean',
                                                 'move_min_sum_std': 'move_min_std',
                                                 'weight_mean_mean': 'weight',
                                                 'quantity_sum_mean': 'steps_day_mean',
                                                   'quantity_sum_std': 'steps_day_std',
                                                'step_min_mean_mean': 'step_min_mean'})
    
    return act_cleaned_part

In [7]:
act_df = google_activities(third_party_csv_path)

In [14]:
act_features_df = add_activity_features(act_df, baseline_csv_path)

In [15]:
act_features_outliers_df = remove_feature_outliers(act_features_df, features_remove_outliers)

In [16]:
act_cleaned_part_day = aggregate_part_day(act_features_outliers_df, 
                       start_date, end_date, 
                       stepday_min, stepday_max, 
                       act_min)

In [20]:
act_cleaned_part = aggregate_part(act_cleaned_part_day, min_days, max_days)

In [21]:
act_cleaned_part

,participant_id,HP_MET_sum_mean,HP_MET_sum_std,HP_step_sum_mean,HP_step_sum_std,HP_sum_mean,HP_sum_std,MET_max_mean,MET_mean,MET_min_mean,MET_std,activities_count,activity_name_count_std,activity_score_mean,activity_score_std,age,bmi,bmr_eq,calories_std,calories_sum_std,datetime_count,distance_sum_mean,gender,height,min_high_sum_mean,min_high_sum_std,min_med_sum_mean,min_med_sum_std,move_min_mean,move_min_std,steps_day_mean,steps_day_std,step_min_mean,step_min_mean_std,weight
0,1007330152,25.249510,17.109876,2.304902,4.994855,25.257843,17.117530,3.830482,1.952180,0.737952,1.374565,18.073529,13.373304,0.694632,0.288340,63,36.468639,1672.702030,1161.912793,482.306254,68,1829.679975,1,162.500000,0.007843,0.032638,25.242157,17.103194,19.129412,18.330909,3133.735294,1783.266277,28.736946,14.901159,96.300003
1,1012027362,35.007182,24.122499,4.936721,8.529630,35.165583,24.253890,3.909289,2.544861,0.851147,1.422501,16.000000,10.007374,0.679922,0.384252,69,29.480843,1245.645000,846.941897,478.641288,123,1976.608415,0,159.500000,0.067615,0.442743,35.030352,24.140013,34.187805,25.337859,3457.747967,2279.685402,46.335363,15.011781,75.000000
2,1012400211,21.986806,15.644690,3.867014,7.358197,21.995486,15.651549,3.840017,2.517336,0.918371,1.352207,15.041667,7.674102,0.546682,0.354808,60,25.207071,1455.847030,795.885192,516.546715,48,1264.373537,1,167.000000,0.008681,0.039945,21.978125,15.637929,19.944792,16.115888,2155.145833,1607.660028,37.721079,13.631218,70.300003
3,1018146705,6.235185,8.964153,0.022222,0.052081,6.235185,8.964153,2.591137,1.370444,0.910585,0.715364,8.777778,4.941488,0.770743,0.314093,69,21.768362,1268.972950,978.052066,398.575227,18,627.483082,0,175.699997,0.000000,0.000000,6.235185,8.964153,2.556481,6.594267,1190.611111,886.027628,11.541981,8.601624,67.199997
4,1020576262,35.635714,20.653036,0.428571,0.887084,35.635714,20.653036,3.692647,2.123241,0.961868,1.347571,16.428571,5.883795,1.073612,0.083555,41,21.123106,1160.622989,1246.058573,96.975968,7,1483.772264,0,158.699997,0.000000,0.000000,35.635714,20.653036,14.026190,9.772585,3107.285714,762.434634,20.009754,7.594312,53.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,9972023164,29.638889,14.260733,2.524306,3.142347,29.823611,14.215693,4.026033,2.320757,0.838275,1.424085,26.708333,10.580865,0.789165,0.195327,48,25.342201,1653.055000,1304.533372,322.886093,24,2288.508494,1,176.000000,0.131250,0.448463,29.561111,14.249309,26.590278,15.424060,3857.000000,1363.403924,36.529167,9.378504,78.500000
2207,9973999327,28.235417,18.898857,1.148750,3.621241,28.236667,18.899391,3.832631,2.204958,0.751635,1.454622,17.362500,9.622289,0.617935,0.286451,63,31.475981,1769.065000,1093.166864,506.749832,80,1728.893265,1,176.000000,0.001458,0.008041,28.233750,18.898379,21.552917,18.843452,2798.262500,1660.882210,30.524605,10.198300,97.500000
2208,9975645275,40.214103,24.989266,7.852564,8.683727,40.241026,24.983573,4.114014,2.398613,0.972234,1.357012,41.538462,22.958714,1.065026,0.177656,54,19.546259,1238.985000,1319.551802,220.113377,13,3159.480012,0,173.000000,0.026923,0.067884,40.187179,24.995142,36.988462,22.598724,5127.461538,2345.650323,38.695196,8.089773,58.500000
2209,9985461404,44.466667,32.161431,2.239815,4.477650,44.692593,32.103831,3.966550,2.227905,0.864332,1.419925,29.277778,12.385027,0.938084,0.204488,67,22.775196,1675.292931,1571.565591,342.577052,18,3658.828932,1,189.399994,0.225926,0.909827,44.240741,32.244609,41.110185,43.990961,5331.666667,3375.414815,28.544516,13.113564,81.699997
